In [110]:
import os
import json
import jsonlines
import time
import pandas as pd
import pickle as pkl
from IPython.display import display

from lkae.utils.data_loading import pkl_dir, load_pkl, load_pkls, root_dir, AuredDataset
from lkae.verification.verify import get_verifier
from lkae.utils.scoring import eval_run_custom_nofile
from lkae.verification.verify import Judge, run_verifier_on_dataset
from lkae.utils.data_loading import AuthorityPost

datasets = load_pkls(pkl_dir)

# possilbe splits: train, dev, train_dev_combined
# (test, all_combined don't have "labels")
split = 'train_dev_combined'

dataset_split = f'English_{split}'
qrel_filename = f'{dataset_split}_qrels.txt'

dataset_variations_dict = datasets[dataset_split]
print(dataset_variations_dict.keys())

dict_keys(['nopre-nam-bio', 'nopre-nam-nobio', 'nopre-nonam-bio', 'nopre-nonam-nobio', 'pre-nam-bio', 'pre-nam-nobio', 'pre-nonam-bio', 'pre-nonam-nobio'])


In [111]:
# ground truth RQ2
gold_file = os.path.join(root_dir, 'data', f'{dataset_split}.jsonl')
gold_list = [line for line in jsonlines.open(gold_file)]

# select a set of variations of the dataset
selected_variations = ["pre-nonam-nobio"]

In [112]:
# load each config and construct its verifier

verifiers = {}

with open('config-compare.json', 'r') as file:
    configs = json.load(file)

    for config in configs['configs']:
        verifier_label = get_verifier(**config)
        verifiers[config['verifier_method']] = verifier_label

verifiers

{'llama3-1-8B': <lkae.verification.models.llama3_azure_ai.Llama3AzureVerifier at 0x1ba8a8d1270>,
 'llama3-1-70B': <lkae.verification.models.llama3_azure_ai.Llama3AzureVerifier at 0x1ba936b0550>,
 'llama3-1-405B': <lkae.verification.models.llama3_azure_ai.Llama3AzureVerifier at 0x1ba95fd4250>}

In [113]:
# compare outputs for different verifiers on one dataset
# first get judgements for all verifiers

# get the dataset
selected_variation = selected_variations[0]
dataset: AuredDataset = dataset_variations_dict[selected_variation]
dataset = dataset[:]
for i, item in enumerate(dataset):
    retrieved_ev = []
    evidences = item["evidence"]
    if evidences is None:
        print(f"skipped {i} because no evidence")
        continue
    for ev in evidences:
        retrieved_ev.append(AuthorityPost(ev.url, ev.post_id, ev.text, 1, 1))
    dataset[i]["retrieved_evidence"] = retrieved_ev

solomon = Judge(
    scale=False,  # ignore scaling, weigh each evidence evenly, except for confidence score given by verifier
    ignore_nei=True,  # ignore NEI predictions
)

results = {}
for verifier_label in verifiers:
    start = time.time()

    run_filename = f'results/{selected_variation}_{verifier_label}.pkl'

    # check if the file already exists from a previous run
    if os.path.exists(run_filename):
        print(f'found {run_filename}, loading from file')
        verification_results = pkl.load(open(run_filename, 'rb'))
    else:
        print(f'running {verifier_label} on {selected_variation}')
        verification_results = run_verifier_on_dataset(
            dataset=dataset,
            verifier=verifiers[verifier_label],
            judge=solomon,
            blind=False,
        )
        pkl.dump(verification_results, open(run_filename, 'wb'))


    macro_f1, strict_macro_f1 = eval_run_custom_nofile(verification_results, gold_list)

    results[verifier_label] = {
        "res_dict": verification_results,
        "time": time.time() - start,
        "macro_f1": macro_f1,
        "strict_f1": strict_macro_f1,
        "settings": {"verifier": verifier_label, "dataset": selected_variation},
    }

pkl.dump(results, open('results/results.pkl', 'wb'))

found results/pre-nonam-nobio_llama3-1-8B.pkl, loading from file
found results/pre-nonam-nobio_llama3-1-70B.pkl, loading from file
found results/pre-nonam-nobio_llama3-1-405B.pkl, loading from file


In [114]:
# print(json.dumps(results, indent=2))

In [115]:
# {
#   "transformers-roberta": {
#     "res_dict": [
#       {
#         "id": "AuRED_142",
#         "label": "REFUTES",
#         "claim": "Naturalization decree in preparation: Lebanese passports for sale !",
#         "predicted_label": "REFUTES",
#         "predicted_evidence": [
#           [
#             "https://twitter.com/LBpresidency",
#             "1555986659279360001",
#             "Statement from Authority Account 'LBpresidency': ''The Information Office of the Presidency of the Republic denies a false news broadcast by the MTV station about Baabda Palace preparing a decree naturalizing 4 000 people and recalls that it had denied yesterday the false information published by the French magazine 'Liberation' about the same fabricated news ''",
#             0.5575303435325623
#           ],
#           [
#             "https://twitter.com/LBpresidency",
#             "1555424541509386240",
#             "Statement from Authority Account 'LBpresidency': ''The Information Office of the Presidency of the Republic: What was published by the French newspaper 'Liberation' about the 'selling' of Lebanese passports to non-Lebanese is false and baseless news '",
#             0.9313378930091858
#           ]
#         ]
#       },
#     ],
#     "time": 1.071157693862915,
#     "macro_f1": 0.27692307692307694,
#     "strict_f1": 0.27692307692307694,
#     "settings": {
#       "verifier": "transformers-roberta",
#       "dataset": "pre-nonam-nobio"
#     }
#   },
# }

In [116]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [117]:
# Create a dictionary to store the comparison data
comparison_data = {}

# Determine the maximum number of evidence pieces across all results
max_evidence = max(
    max(len(item['predicted_evidence']) for item in result['res_dict'])
    for result in results.values()
)

# clamp to max 5 evidence pieces
max_evidence = min(max_evidence, 5)

# Iterate through each result in the results dictionary
for verifier_label, result in results.items():
    for item in result['res_dict']:
        item_id = item['id']
        if item_id not in comparison_data:
            comparison_data[item_id] = {
                'id': item['id'],
                'claim': item['claim'],
                'label': item['label'],
            }
        
        comparison_data[item_id][f'{verifier_label}-pred_label'] = item['predicted_label']
        
        # Add evidence columns with the new format
        for i in range(max_evidence):
            if i < len(item['predicted_evidence']):
                ev = item['predicted_evidence'][i]
                comparison_data[item_id][f'{verifier_label}-ev_{i+1}'] = f"({ev[3]:.1f}) {ev[2]} "
            else:
                comparison_data[item_id][f'{verifier_label}-ev_{i+1}'] = ''

# Create a DataFrame from the comparison data
df_comparison = pd.DataFrame(list(comparison_data.values()))

# Define the column order
columns = ['id', 'claim', 'label']
for verifier in results.keys():
    columns.append(f'{verifier}-pred_label')

for verifier in results.keys():
    for i in range(max_evidence):
        columns.append(f'{verifier}-ev_{i+1}')

# Reorder the DataFrame columns
df_comparison = df_comparison[columns]

# Add a column to indicate if predicted labels match
def labels_match(row):
    predicted_labels = [row[f'{verifier}-pred_label'] for verifier in results.keys()]
    return 'Yes' if len(set(predicted_labels)) == 1 else 'No'

df_comparison['Predicted Labels Match'] = df_comparison.apply(labels_match, axis=1)

# Move the 'Predicted Labels Match' column to the end
cols = df_comparison.columns.tolist()
cols = cols[:-1] + cols[-1:]
df_comparison = df_comparison[cols]

# Display the DataFrame
# display(df_comparison)

# Optionally, save the DataFrame to a CSV file for easier viewing in spreadsheet software
df_comparison.to_csv('results/verifier_comparison_detailed.csv', index=False)

# Print summary statistics
print("\nSummary Statistics:")
for verifier_label, result in results.items():
    print(f"\n{verifier_label}:")
    print(f"  Macro F1: {result['macro_f1']:.4f}")
    print(f"  Strict F1: {result['strict_f1']:.4f}")
    print(f"  Time: {result['time']:.2f} seconds")

# Print the percentage of matching predictions
matching_percentage = (df_comparison['Predicted Labels Match'] == 'Yes').mean() * 100
print(f"\nPercentage of matching predictions: {matching_percentage:.2f}%")


Summary Statistics:

llama3-1-8B:
  Macro F1: 0.8145
  Strict F1: 0.8145
  Time: 0.00 seconds

llama3-1-70B:
  Macro F1: 0.9433
  Strict F1: 0.9433
  Time: 0.00 seconds

llama3-1-405B:
  Macro F1: 0.9853
  Strict F1: 0.9853
  Time: 0.00 seconds

Percentage of matching predictions: 80.00%


In [118]:
v = 'llama3-1-405B'
f = ['id', 'label', f'{v}-pred_label', 'claim']
for i in range(max_evidence):
    f.append(f'{v}-ev_{i+1}')
df_comparison_filtered = df_comparison[f]
d = df_comparison_filtered.loc[df_comparison_filtered[f'{v}-pred_label'] != df_comparison_filtered['label']]
d.columns = ['id', 'actual label', 'predicted label', 'claim', 'evidence 1', 'evidence 2', 'evidence 3', 'evidence 4', 'evidence 5']
print(len(d))
display(d)

2


,id,actual label,predicted label,claim,evidence 1,evidence 2,evidence 3,evidence 4,evidence 5
10,AuRED_157,REFUTES,NOT ENOUGH INFO,God is Great and praise be to God The release of Sheikh Salman Al-Awda from the deadly prisons of Al-Salool May God hasten the release of the captivity of all his brothers scholars and oppressed oppressed people,(-0.8) Statement from Authority Account 'aalodah': ' Salman Al-Awda Conference',(0.8) Statement from Authority Account 'aalodah': ' Alia Al-Hathloul did well in reminding the rest of the detainees especially Islamists like Sheikh Salman Al-Awda in order to ignore his attempt to separate prisoners of conscience based on their attitudes Prisoner of conscience Prisoner of conscience! There is no difference between a liberal opinion and an Islamic opinion aalodah',(0.0) Statement from Authority Account 'aalodah': ' After a thousand and one nights I can finally see the face of my dear sister Loujain but I cannot say that she is free Also I cannot forget that Al-Awda Al-Sadhan Al-Rabiah and others are still subjected to great injustice Life lesson: No right is lost there are demands behind it ',(-0.8) Statement from Authority Account 'aalodah': 'The day after tomorrow Friday wait for us moslehun Salman Return Conference ',(0.8) Statement from Authority Account 'aalodah': ' The virtual international conference 'Contemporary Reformers: Salman Al-Awda as a Model' with the participation of a global elite of academics jurists and politicians ️February 12 and 13 2021 Conference program and broadcast links Salman Al-Awda Conference''
65,AuRED_159,REFUTES,NOT ENOUGH INFO,News about the death of Sheikh Abdul Rahman Al-Sudais Friday of Rage 25 September,(0.0) Statement from Authority Account 'ReasahAlharmain': 'General President of General Managers: The pilgrim must find the finest and best services Presidency of The Haramayn Affairs ',(0.0) Statement from Authority Account 'ReasahAlharmain': 'The President-General praises the valor and professionalism of the security men and the heroic work they did in arresting terrorist elements and exposing their failed and miserable plans Presidency of The Haramain Affairs ',(0.0) Statement from Authority Account 'ReasahAlharmain': 'In commemoration of the heroic role of the martyrs of duty the General President directs the appointment of one of their sons ',(0.0) Statement from Authority Account 'ReasahAlharmain': 'The President-General: Our soldiers made the highest sacrifices to uphold the word of monotheism and defend the sanctities of our dear country ',(0.0) Statement from Authority Account 'ReasahAlharmain': 'The President-General discusses preparations to receive Umrah pilgrims and worshipers with the field executive team for the Umrah season Presidency of The Haramayn Affairs '


In [119]:
v = 'llama3-1-70B'
f = ['id', 'label', f'{v}-pred_label', 'claim']
for i in range(max_evidence):
    f.append(f'{v}-ev_{i+1}')
df_comparison_filtered = df_comparison[f]
d = df_comparison_filtered.loc[df_comparison_filtered[f'{v}-pred_label'] != df_comparison_filtered['label']]
d.columns = ['id', 'actual label', 'predicted label', 'claim', 'evidence 1', 'evidence 2', 'evidence 3', 'evidence 4', 'evidence 5']
print(len(d))
display(d.loc[[16,67]])
display(d)

6


,id,actual label,predicted label,claim,evidence 1,evidence 2,evidence 3,evidence 4,evidence 5
16,AuRED_124,SUPPORTS,NOT ENOUGH INFO,Saudi Arabia bears the costs of electronic services for the Haya visa for the World Cup in Qatar,(0.9) Statement from Authority Account 'KSAMOFA': ' Cabinet: Approval for the state to bear the costs of electronic services - related to the electronic visa services platform at the Ministry of Foreign Affairs - that are collected in exchange for issuing an entry visa to the Kingdom for holders of the fan card (Haya) for the Qatar 2022 World Cup event ',(-0.9) Statement from Authority Account 'spagov': ' Cabinet: Approval for the state to bear the costs of electronic services - related to the electronic visa services platform at the Ministry of Foreign Affairs - that are collected in exchange for issuing an entry visa to the Kingdom for holders of the fan card (Haya) for the Qatar 2022 World Cup event SPA ',,,
67,AuRED_041,REFUTES,SUPPORTS,Vaccination conspiracy Do we agree that even if there is a question mark Zain Telecommunications Company Kuwait has committed a major mistake that must be corrected quickly !,(-0.8) Statement from Authority Account 'ZainKuwait': 'Vaccination can only be a conspiracy against infection Let us all conspire against the epidemic in order to return to our old life which we miss greatly at all levels Make sense of it and trust taste ',,,,


,id,actual label,predicted label,claim,evidence 1,evidence 2,evidence 3,evidence 4,evidence 5
2,AuRED_085,REFUTES,NOT ENOUGH INFO,Saudi Arabia evacuated 10 students from China in a plane that could accommodate 500 passengers and did not evacuate 170 Yemeni students with it while participating with 100 warplanes to bomb Yemen May God protect the Sultanate of Oman for sending a private plane to the Yemeni students stranded in China And what a good neighbor Oman ️ co OfDSIoD5ry,(-0.8) Statement from Authority Account 'YSUCORG': ' The Yemeni Embassy in China announces the start of disbursing financial aid to all Yemeni students and their families residing in the Chinese city of Wuhan which is witnessing the spread of the Corona virus in the coming days ',(-0.8) Statement from Authority Account 'YSUCORG': 'Statement by the administrative body of the Yemeni Students Union in China regarding the conditions of students in Wuhan ',(-0.8) Statement from Authority Account 'Yemen PM': 'The Yemeni Embassy in China announces the start of disbursing financial aid to all Yemeni students and their families residing in the Chinese city of Wuhan which is witnessing the spread of the Corona virus in the coming days ',(0.0) Statement from Authority Account 'Yemen PM': 'Deputy Prime Minister Dr Salem Al-Khanbashi chairs an expanded meeting to study precautionary arrangements to prevent the spread of the Coronavirus to Yemen noting the directives of Prime Minister Dr Maeen Abdulmalik to form an emergency committee and take all measures to protect students and the Yemeni community in China ',(0.0) Statement from Authority Account 'Yemen PM': 'Prime Minister Dr Maeen Abdulmalik makes a phone call with the Chargé d'Affaires of the Yemeni Embassy in China Ahmed Jaber to check on the conditions of the students and the Yemeni community stressing that the government under the directives of the President of the Republic will provide in coordination with the Chinese authorities all support to maintain their safety as a result of the outbreak of the Corona virus t co UK2xfjBMYk'
10,AuRED_157,REFUTES,SUPPORTS,God is Great and praise be to God The release of Sheikh Salman Al-Awda from the deadly prisons of Al-Salool May God hasten the release of the captivity of all his brothers scholars and oppressed oppressed people,(-0.8) Statement from Authority Account 'aalodah': ' Salman Al-Awda Conference',(0.0) Statement from Authority Account 'aalodah': ' Alia Al-Hathloul did well in reminding the rest of the detainees especially Islamists like Sheikh Salman Al-Awda in order to ignore his attempt to separate prisoners of conscience based on their attitudes Prisoner of conscience Prisoner of conscience! There is no difference between a liberal opinion and an Islamic opinion aalodah',(0.8) Statement from Authority Account 'aalodah': ' After a thousand and one nights I can finally see the face of my dear sister Loujain but I cannot say that she is free Also I cannot forget that Al-Awda Al-Sadhan Al-Rabiah and others are still subjected to great injustice Life lesson: No right is lost there are demands behind it ',(-0.8) Statement from Authority Account 'aalodah': 'The day after tomorrow Friday wait for us moslehun Salman Return Conference ',(0.0) Statement from Authority Account 'aalodah': ' The virtual international conference 'Contemporary Reformers: Salman Al-Awda as a Model' with the participation of a global elite of academics jurists and politicians ️February 12 and 13 2021 Conference program and broadcast links Salman Al-Awda Conference''
16,AuRED_124,SUPPORTS,NOT ENOUGH INFO,Saudi Arabia bears the costs of electronic services for the Haya visa for the World Cup in Qatar,(0.9) Statement from Authority Account 'KSAMOFA': ' Cabinet: Approval for the state to bear the costs of electronic services - related to the electronic visa services platform at the Ministry of Foreign Affairs - that are collected in exchange for issuing an entry visa to the Kingdom for holders of

In [120]:
v = 'llama3-1-8B'
f = ['id', 'label', f'{v}-pred_label', 'claim']
for i in range(max_evidence):
    f.append(f'{v}-ev_{i+1}')
df_comparison_filtered = df_comparison[f]
d = df_comparison_filtered.loc[df_comparison_filtered[f'{v}-pred_label'] != df_comparison_filtered['label']]
d.columns = ['id', 'actual label', 'predicted label', 'claim', 'evidence 1', 'evidence 2', 'evidence 3', 'evidence 4', 'evidence 5']
print(len(d))
display(d.loc[[90]])
display(d)

21


,id,actual label,predicted label,claim,evidence 1,evidence 2,evidence 3,evidence 4,evidence 5
90,AuRED_083,REFUTES,SUPPORTS,Libya - Sources of the event: News of the arrival of an American delegation to Benghazi to discuss efforts for a solution,(-0.9) Statement from Authority Account 'USEmbassyLibya': ' AlHadath AlHadath There is no American delegation heading to Benghazi This rumor is not true ',,,,


,id,actual label,predicted label,claim,evidence 1,evidence 2,evidence 3,evidence 4,evidence 5
0,AuRED_014,REFUTES,SUPPORTS,' Urgent Ramallah Ministry of Health spokesman Kamal Al-Shakhra: We received 2 000 doses of the American 'Moderna' Corona vaccine and this batch will be designated for President Abbas the Fatah Central Committee and VIPs ',(-0.8) Statement from Authority Account 'palestine moh': 'Pictures from the launch of the vaccination campaign against the Coronavirus starting with the medical and health teams in Bethlehem vaccine Palestine COVID19 ',(-0.8) Statement from Authority Account 'palestine moh': 'The Ministry added that the second group that will receive the vaccine is the elderly over the age of 60 and those with chronic diseases because if they are infected with the virus they will be more susceptible to serious symptoms vaccine modernavaccine palestine COVID19',(0.9) Statement from Authority Account 'palestine moh': 'Yesterday the Ministry of Health announced that it had received 2 000 doses of Moderna vaccine stressing that the priority in receiving anti-Coronavirus vaccinations is for personnel working in the health sector as they are the group who have the most contact with patients and are most at risk of infection with the Corona virus modernavaccine palestine COVID19',(-0.9) Statement from Authority Account 'palestine moh': 'The Minister of Health Dr May Alkaila today Tuesday the campaign where she received the vaccine at Hogochaffer Hospital in Turmus Aya while the medical staff working in the intensive care rooms there were vaccinated with the Moderna vaccine modernavaccine palestine COVID19',
3,AuRED_089,REFUTES,SUPPORTS,The Corona epidemic has reached the Emirates with 75 cases in Abu Dhabi and 63 cases in Dubai A warning to refrain from traveling to the Emirates in order to preserve safety and not transmit the epidemic May God protect Muslims everywhere,(-0.9) Statement from Authority Account 'WHOEMRO': 'Today WHO confirmed the emergence of the first cases of the new Coronavirus in the Eastern Mediterranean Region in the United Arab Emirates Following mohapuae's confirmation on January 29 4 members of the same family from the Chinese city of Wuhan arrived in the UAE at the beginning of January 2020 and were admitted to the hospital after they were confirmed infected with the Corona virus ',(0.0) Statement from Authority Account 'mohapuae': 'A new case of infection with the new Corona virus was recorded for a person coming from the city of Wuhan in the People's Republic of China Thus the number of cases discovered in the country is only five since the outbreak of the disease to date New Coronavirus Coronavirus ',(0.0) Statement from Authority Account 'mohapuae': 'Four people from a Chinese family were infected with the new Corona virus all of whom are in stable condition and were contained according to internationally approved precautionary measures ',(-0.9) Statement from Authority Account 'mohapuae': 'The general health situation does not cause concern A case of infection with the new Coronavirus has been diagnosed for people from the same family coming from the city of Wuhan in the People's Republic of China The health condition of the injured is stable and under medical observation Ministry of Health and Community Prevention ',(-0.9) Statement from Authority Account 'NCEMAUAE': ' A new case of infection with the new Corona virus was recorded for a person coming from the city of Wuhan in the People's Republic of China thus bringing the number of cases discovered in the country to only five since the outbreak of the disease until today New Corona Virus Corona Virus'
4,AuRED_135,REFUTES,NOT ENOUGH INFO,The official spokesman for the Football Association: Al-Ahly sent a letter to the Football Association announcing its refusal to wear Zamalek in the full blue uniform during the Egyptian Super match based on its request,(-0.8) Statement from Authority Account 'AlAhlyTV': 'Islam Al-Shater 'warns' 